# 8. 프로젝트 - Movielens 영화 추천 실습

## 8-1. 데이터 준비와 전처리

In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count'}, inplace=True)

In [4]:
ratings['count']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
#데이터셋 병합. merge 사용.
result = pd.merge(ratings, movies, on = 'movie_id')
result.rename(columns={'count' : 'rating'}, inplace = True)
print(len(result))
result.head(len(result))

836478


,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,978298413,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,978220179,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,978199279,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,978158471,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
836473,5851,3607,5,957756608,One Little Indian (1973),Comedy|Drama|Western
836474,5854,3026,4,958346883,Slaughterhouse (1987),Horror
836475,5854,690,3,957744257,"Promise, The (Versprechen, Das) (1994)",Romance
836476,5938,2909,4,957273353,"Five Wives, Three Secretaries and Me (1998)",Documentary


merge후 rating이 count로 변경되어있어 rename사용하여 변경.

## 8-2 분석

In [7]:
#result에 있는 유니크한 영화 개수
result['movie_id'].nunique()

3628

In [8]:
#result에 있는 유니크한 사용자 수 
result['user_id'].nunique()

6039

In [9]:
result['title'] = result['title'].str.lower()
result.head()

,user_id,movie_id,rating,timestamp,title,genre
0,1,1193,5,978300760,one flew over the cuckoo's nest (1975),Drama
1,2,1193,5,978298413,one flew over the cuckoo's nest (1975),Drama
2,12,1193,4,978220179,one flew over the cuckoo's nest (1975),Drama
3,15,1193,4,978199279,one flew over the cuckoo's nest (1975),Drama
4,17,1193,5,978158471,one flew over the cuckoo's nest (1975),Drama


In [10]:
#가장 인기 있는 영화 30개(평점순)#가장 인기 있는 영화 30개(평점순)
movie_count_rating = result.groupby(['title'], as_index=False).mean()
movie_count_rating.sort_values(by='rating', ascending=False).head(30)

,title,user_id,movie_id,rating,timestamp
1280,"gate of heavenly peace, the (1995)",1948.666667,787.0,5.000000,9.741198e+08
1186,follow the bitch (1998),2869.000000,1830.0,5.000000,9.724390e+08
1625,identification of a woman (identificazione di ...,5948.000000,1360.0,5.000000,1.016564e+09
2854,schlafes bruder (brother of sleep) (1995),1915.000000,989.0,5.000000,9.746939e+08
782,criminal lovers (les amants criminels) (1999),850.000000,3800.0,5.000000,9.753594e+08
765,country life (1994),1631.000000,687.0,5.000000,9.747256e+08
1206,foreign student (1994),3618.000000,572.0,5.000000,9.671202e+08
1991,lured (1947),5313.000000,3656.0,5.000000,9.609204e+08
246,"baby, the (1973)",46.000000,3280.0,5.000000,9.779795e+08
1871,late bloomers (1996),4682.000000,1553.0,5.000000,9.645669e+08


In [11]:
#가장 인기 있는 영화 30개(리뷰순)
movie_count_review = result.groupby('title')['user_id'].count()
movie_count_review.sort_values(ascending=False).head(30)

title
american beauty (1999)                                   3211
star wars: episode iv - a new hope (1977)                2910
star wars: episode v - the empire strikes back (1980)    2885
star wars: episode vi - return of the jedi (1983)        2716
saving private ryan (1998)                               2561
terminator 2: judgment day (1991)                        2509
silence of the lambs, the (1991)                         2498
raiders of the lost ark (1981)                           2473
back to the future (1985)                                2460
matrix, the (1999)                                       2434
jurassic park (1993)                                     2413
sixth sense, the (1999)                                  2385
fargo (1996)                                             2371
braveheart (1995)                                        2314
men in black (1997)                                      2297
schindler's list (1993)                                  2257
pr

## 8-3 내가 선호하는 영화 5가지 추가

In [12]:
# 본인이 좋아하시는 아티스트 데이터로 바꿔서 추가하셔도 됩니다! 단, 이름은 꼭 데이터셋에 있는 것과 동일하게 맞춰주세요. 
my_favorite = ['toy story (1995)' , 'godfather, the (1972)' ,'matrix, the (1999)' ,'men in black (1997)' ,'terminator 2: judgment day (1991)']

# 'zimin'이라는 user_id가 위 아티스트의 노래를 30회씩 들었다고 가정하겠습니다.
my_playlist = pd.DataFrame({'user_id': ['minkyu']*5, 'title': my_favorite, 'rating':[5]*5})

if not result.isin({'user_id':['minkyu']})['user_id'].any():  # user_id에 'zimin'이라는 데이터가 없다면
    result = result.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

result.tail(10)       # 잘 추가되었는지 확인해 봅시다.

,user_id,movie_id,rating,timestamp,title,genre
836473,5851,3607.0,5,9.577566e+08,one little indian (1973),Comedy|Drama|Western
836474,5854,3026.0,4,9.583469e+08,slaughterhouse (1987),Horror
836475,5854,690.0,3,9.577443e+08,"promise, the (versprechen, das) (1994)",Romance
836476,5938,2909.0,4,9.572734e+08,"five wives, three secretaries and me (1998)",Documentary
836477,5948,1360.0,5,1.016564e+09,identification of a woman (identificazione di ...,Drama
0,minkyu,NaN,5,NaN,toy story (1995),NaN
1,minkyu,NaN,5,NaN,"godfather, the (1972)",NaN
2,minkyu,NaN,5,NaN,"matrix, the (1999)",NaN
3,minkyu,NaN,5,NaN,men in black (1997),NaN
4,minkyu,NaN,5,NaN,terminator 2: judgment day (1991),NaN


In [13]:
#사용하지 않는 colmes 제거
result = result.drop(['movie_id', 'timestamp', 'genre'], axis = 1)
result

,user_id,rating,title
0,1,5,one flew over the cuckoo's nest (1975)
1,2,5,one flew over the cuckoo's nest (1975)
2,12,4,one flew over the cuckoo's nest (1975)
3,15,4,one flew over the cuckoo's nest (1975)
4,17,5,one flew over the cuckoo's nest (1975)
...,...,...,...
0,minkyu,5,toy story (1995)
1,minkyu,5,"godfather, the (1972)"
2,minkyu,5,"matrix, the (1999)"
3,minkyu,5,men in black (1997)


In [14]:
# 고유한 유저, title를 찾아내는 코드
user_unique = result['user_id'].unique()
title_unique = result['title'].unique()
print(user_unique)
# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
title_to_idx = {v:k for k,v in enumerate(title_unique)}

# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['minkyu'])    # 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(title_to_idx['toy story (1995)'])

[1 2 12 ... 2908 4486 'minkyu']
6039
40


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_result = result['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_result) == len(result):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    result['user_id'] = temp_user_result   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_title_data = result['title'].map(title_to_idx.get).dropna()
if len(temp_title_data) == len(result):
    print('title column indexing OK!!')
    result['title'] = temp_title_data
else:
    print('title column indexing Fail!!')

result

user_id column indexing OK!!
title column indexing OK!!


,user_id,rating,title
0,0,5,0
1,1,5,0
2,2,4,0
3,3,4,0
4,4,5,0
...,...,...,...
0,6039,5,40
1,6039,5,607
2,6039,5,124
3,6039,5,175


## 8-4 CSR matrix 생성

In [16]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = result['user_id'].nunique()
num_title = result['title'].nunique()

csr_data = csr_matrix((result.rating, (result.user_id, result.title)), shape= (num_user, num_title))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 8-5 als_model = AlternatingLeastSquares 모델 구성

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 8-6 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악

In [21]:
minkyu, toy_story = user_to_idx['minkyu'], title_to_idx['toy story (1995)']
minkyu_vector, toy_story_vector = als_model.user_factors[minkyu], als_model.item_factors[toy_story]

print('슝=3')

슝=3


In [22]:
minkyu_vector

array([ 8.30754459e-01,  6.66141808e-01,  1.55681491e-01,  2.87987769e-01,
       -3.39669660e-02, -2.71115571e-01, -2.92442709e-01,  4.11022007e-01,
       -1.58857480e-01, -5.90659022e-01, -6.88401222e-01,  3.41909438e-01,
       -4.58592027e-01,  2.69315869e-01,  8.76669586e-02,  5.20441115e-01,
        4.66635704e-01,  5.83151519e-01,  6.08485520e-01,  1.90828457e-01,
        8.46778214e-01,  4.21287268e-01, -3.97698373e-01,  3.74637306e-01,
       -4.03709412e-01,  7.18122721e-02,  1.21896425e-02, -7.35871792e-01,
        5.86605489e-01,  1.84217140e-01, -8.34954262e-01, -3.86185855e-01,
        9.74300265e-01,  1.22447121e+00, -1.35572815e+00,  3.04685265e-01,
       -3.75058800e-01,  8.94248545e-01,  5.01166582e-01, -1.24116812e-03,
        7.25893378e-02,  4.47468221e-01,  7.07739964e-02, -9.40048769e-02,
        1.09427166e+00, -4.20362391e-02, -5.14249384e-01, -6.67016327e-01,
       -4.24460322e-01,  5.18789828e-01, -4.43112969e-01,  1.85013600e-02,
       -2.14581147e-01, -

In [23]:
toy_story_vector

array([-2.1042703e-02, -4.8656641e-03,  5.6936853e-03,  3.1455315e-03,
        4.2957081e-03,  2.2980569e-02,  1.7340260e-02,  1.4622633e-02,
        1.1295003e-02, -1.1456530e-02, -2.8645759e-02, -9.9786445e-03,
       -2.4076488e-02,  3.5103440e-02,  2.7476668e-02,  3.1075787e-02,
       -2.1321977e-02, -9.6276391e-04,  5.3252332e-02,  7.1005770e-03,
        3.2885835e-02,  3.2590952e-02,  6.8365932e-03, -2.2238882e-03,
        5.3257607e-03, -1.4073673e-02,  6.0764705e-03, -2.0527145e-02,
        9.9759949e-03,  2.7874481e-02,  2.3228714e-02, -2.5844376e-05,
        2.7996007e-02,  3.5686929e-02, -2.4271635e-02,  2.1326490e-02,
       -2.2368211e-02,  2.8925112e-02,  1.9363878e-02, -6.2817940e-03,
        7.7064442e-03, -8.8722091e-03, -3.4751308e-03, -7.5359289e-03,
        6.7322170e-03,  3.5604634e-03,  9.3251811e-03,  8.5797813e-03,
        2.1876959e-02,  2.9543813e-02, -1.2752316e-02, -2.4409932e-03,
       -1.6310403e-02, -1.7130427e-02, -1.7089207e-03,  2.5859170e-02,
      

In [24]:
np.dot(minkyu_vector, toy_story_vector)

0.5121031

In [25]:
song_of_freedom = title_to_idx['song of freedom (1936)']
song_of_freedom_vector = als_model.item_factors[song_of_freedom]
np.dot(minkyu_vector, song_of_freedom_vector)

-0.00042835774

## 8-7 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [30]:
favorite_title = 'godfather, the (1972)'
title_id = title_to_idx[favorite_title]
similar_title = als_model.similar_items(title_id, N=15)
similar_title

[(607, 1.0000001),
 (380, 0.9635351),
 (435, 0.49877742),
 (128, 0.4412381),
 (3531, 0.36430645),
 (3517, 0.35746807),
 (601, 0.3536402),
 (269, 0.34395254),
 (3150, 0.3429832),
 (432, 0.33782357),
 (57, 0.3258748),
 (3499, 0.31209934),
 (3601, 0.3114027),
 (753, 0.2907554),
 (361, 0.27983722)]

In [31]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_title = {v:k for k,v in title_to_idx.items()}
[idx_to_title[i[0]] for i in similar_title]

['godfather, the (1972)',
 'godfather: part ii, the (1974)',
 'godfather: part iii, the (1990)',
 'french connection, the (1971)',
 'aiqing wansui (1994)',
 'paralyzing fear: the story of polio in america, a (1998)',
 'citizen kane (1941)',
 'goodfellas (1990)',
 'boys, the (1997)',
 'taxi driver (1976)',
 'untouchables, the (1987)',
 'simon sez (1999)',
 'condition red (1995)',
 'dog day afternoon (1975)',
 'casablanca (1942)']

In [27]:
def get_similar_title(title_name: str):
    title_id = title_to_idx[title_name]
    similar_title = als_model.similar_items(title_id)
    similar_title = [idx_to_title[i[0]] for i in similar_title]
    return similar_title

print("슝=3")

슝=3


In [32]:
get_similar_title('toy story (1995)')

['toy story (1995)',
 'toy story 2 (1999)',
 "bug's life, a (1998)",
 'babe (1995)',
 'groundhog day (1993)',
 'aladdin (1992)',
 'lion king, the (1994)',
 "there's something about mary (1998)",
 'beauty and the beast (1991)',
 'hercules (1997)']

## 8-8 내가 좋아할만한 영화 추천

In [33]:
user = user_to_idx['minkyu']
# recommend에서는 user*item CSR Matrix를 받습니다.
title_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
title_recommended

[(380, 0.5522106),
 (107, 0.5491893),
 (50, 0.43806264),
 (62, 0.4353689),
 (87, 0.3809426),
 (110, 0.33070385),
 (141, 0.3193211),
 (64, 0.29607266),
 (4, 0.29060575),
 (200, 0.2871673),
 (44, 0.28184748),
 (435, 0.2723791),
 (117, 0.26770166),
 (670, 0.26304656),
 (375, 0.25779745),
 (60, 0.25675762),
 (48, 0.25297773),
 (126, 0.24081641),
 (75, 0.23086986),
 (22, 0.2237291)]

In [34]:
[idx_to_title[i[0]] for i in title_recommended]

['godfather: part ii, the (1974)',
 'jurassic park (1993)',
 'toy story 2 (1999)',
 'total recall (1990)',
 'braveheart (1995)',
 'groundhog day (1993)',
 'fugitive, the (1993)',
 'star wars: episode vi - return of the jedi (1983)',
 "bug's life, a (1998)",
 'terminator, the (1984)',
 'star wars: episode iv - a new hope (1977)',
 'godfather: part iii, the (1990)',
 'star wars: episode v - the empire strikes back (1980)',
 'galaxy quest (1999)',
 'face/off (1997)',
 'star wars: episode i - the phantom menace (1999)',
 'saving private ryan (1998)',
 'shakespeare in love (1998)',
 'hunt for red october, the (1990)',
 'back to the future (1985)']

In [35]:
rihanna = title_to_idx['face/off (1997)']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [36]:
[(idx_to_title[i[0]], i[1]) for i in explain[1]]

[('matrix, the (1999)', 0.10090693764530762),
 ('men in black (1997)', 0.0730274088347638),
 ('terminator 2: judgment day (1991)', 0.06390762287782187),
 ('godfather, the (1972)', 0.05586492175992225),
 ('toy story (1995)', -0.039025088142632336)]

matrix가 높은 기여